### Лера Бунтякова

Сложные слова для разбора:
- слова с дефисами: мышь-соня, диван-кровать, Тянь-Шань, кресло-качалка (в итоге я вручную их склеивала по дефису, потому что задание было не про токенизацию, а про определение частей речи)
- аббревиатуры: МФТИ, ВГИК (аббревиатуры можно принять за глаголы)
- омонимы: вина (конечно, надо было взять омоним, у которого в разборе будут разные части речи)
- имена собственные: Лидделл, Волков

- NOUN - существительные
- PRON - местоимения 
- VERB - все формы глагола 
- ADVB - наречия и предикативы
- ADJ - все прилагательные 
- CONJ - союзы 
- PRCL - частицы 
- COMP - сравнительные степени наречий и прилагательных 
- PRTF - полные причастия 
- PRTS - краткие причастия
- GRND - деепричастия
- NUMR - числительные
- PREP - предлоги

Я взяла сокращенный тегсет pymorphy с надеждой, что он будет универсальным. Он не был универсальным.

In [1]:
import re

In [2]:
import pandas

In [3]:
true_df = pandas.read_csv('alice.csv')

In [4]:
with open ('alice.txt', 'r') as f:
    text = f.read()

In [5]:
def tokenisation(text): # так можно сделать, потому что я точно знаю, что других токенов там нет
    tokens = re.findall('[а-яА-Я]+-*[а-яА-Я]*', text)
    return tokens

In [6]:
tokens = tokenisation(text)

## Pymorphy

In [7]:
import pymorphy2

In [8]:
morph = pymorphy2.MorphAnalyzer()

In [9]:
def norm_pymorphy(pos):
    if pos == 'ADJF' or pos == 'ADJS':
        return 'ADJ'
    elif pos == 'INFN':
        return 'VERB'
    elif pos == 'PRED':
        return 'ADVB'
    else:
        return pos

In [10]:
t = 0
f = 0
for index, row in true_df.iterrows():
    token = row['token']
    true_pos = row['pos']
    pos = norm_pymorphy(morph.parse(token)[0].tag.POS)
    if str(pos) == true_pos:
        t += 1
    else:
        f += 1
        
print('accuracy: ', t/(t+f))

accuracy:  0.8048780487804879


## Spacy

In [11]:
import spacy
nlp = spacy.load("ru_core_news_lg")

In [12]:
def norm_spacy(pos):
    if pos == 'ADP':
        return 'PREP'
    elif pos == 'CCONJ' or pos == 'SCONJ':
        return 'CONJ'
    elif pos == 'PROPN':
        return 'NOUN'
    elif pos == 'DET':
        return 'PRON'
    elif pos == 'AUX':
        return 'VERB'
    elif pos == 'PART':
        return 'PRCL'
    elif pos == 'NUM':
        return 'NUMR'
    elif pos == 'ADV':
        return 'ADVB'
    else:
        return pos

In [13]:
doc = nlp(text)

In [14]:
t = 0
f = 0
i = 0
for index, row in true_df.iterrows():
    token = row['token']
    true_pos = row['pos']
    
    pos = doc[i].pos_
    while pos in ['SPACE', 'PUNCT', ]:
        i +=1
        pos = doc[i].pos_
    
    text = doc[i].text
    pos = norm_spacy(doc[i].pos_)
    if doc[i+1].text == '-':
        text = doc[i].text + '-' + doc[i+2].text
        i += 2
    i += 1
    if str(pos) == true_pos:
        t += 1
    else:
        f += 1
        
print('accuracy: ', t/(t+f))

accuracy:  0.8414634146341463


## Mystem

In [15]:
# тут мог быть мой код, но я уехала в горы

## Чанкер (с spacy)

не + глагол

In [16]:
for i in range(len(doc)):
    if doc[i].text == 'не' and doc[i+1].pos_ == 'VERB':
        print(doc[i].text, doc[i+1].text)

не увидела
не вижу
не знала
не переходить
не нашелся


прилагательное + существительное

In [17]:
for i in range(len(doc)):
    if doc[i].pos_ == 'ADJ' and (doc[i+1].pos_ == 'NOUN' or doc[i+1].pos_ == 'PROPN'):
        print(doc[i].text, doc[i+1].text)

Мартовский Заяц
Бедная Мышь
большое кресло
Мартовский Заяц
Мартовский Заяц
Мартовский Заяц
Мартовский Заяц


наречие + глагол

In [18]:
for i in range(len(doc)):
    if doc[i].pos_ == 'ADV' and doc[i+1].pos_ == 'VERB':
        print(doc[i].text, doc[i+1].text)

бодро предложил
здесь нет
слишком обросла
широко открыл


## Задание 5

In [19]:
# тут тоже мог быть мой код, но я уехала в горы